<a href="https://colab.research.google.com/github/EnzoEmir/Plantas_Rede_Conv/blob/main/pranta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Nonsegmented.zip -d /content/Nonsegmented

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/544.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/102.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/397.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/507.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/134.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/246.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/194.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/472.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/425.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/115.png  
  inflating: /content/Nonsegmented/Nonsegmented/Small-flowered Cranesbill/571.png  
  inflating: /con

### Importação de bibliotecas necessárias
### Utilizamos bibliotecas para manipulação de imagens, construção de modelos, e treinamento em TensorFlow/Keras.

In [ ]:
import tensorflow as tf
import pathlib


from tensorflow import keras
from keras import layers
from keras import Sequential
from keras import optimizers
from keras import Input
from tensorflow.keras.optimizers import Adam

### Diretório onde as imagens estão armazenadas
### Estratégia: Usamos o pathlib para facilitar a manipulação do caminho das imagens.

In [ ]:
data_dir = pathlib.Path("/content/Nonsegmented/Nonsegmented")

### Contagem de imagens para confirmar o carregamento correto do dataset


In [ ]:
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)

5544


### Definindo os parâmetros de processamento do dataset
### Tamanho das imagens foi escolhido como 290x290 para balancear entre qualidade e desempenho.

    

In [ ]:
batch_size = 32
img_height = 290
img_width = 290

### Criação do dataset de treinamento e validação
### Estratégia: Divisão 80/20 para garantir um conjunto balanceado, minimizando o risco de overfitting.
### Seed fixada para consistência entre execuções.

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 5544 files belonging to 12 classes.
Using 4436 files for training.
Found 5544 files belonging to 12 classes.
Using 1108 files for validation.


### Extração dos nomes das classes para fins de monitoramento e debug


In [ ]:
class_names = train_ds.class_names
print(class_names)

['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherd’s Purse', 'Small-flowered Cranesbill', 'Sugar beet']


### Otimização de carregamento do dataset para melhorar o desempenho durante o treinamento
### Estratégia: Uso de cache para evitar recarregamentos e prefetch para paralelizar processamento.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Número de classes para o modelo
### Definição da camada de entrada


In [ ]:
num_classes = len(class_names)
input_layer = Input(shape=(img_height, img_width, 3))

### Camadas de augmentação de dados
### Estratégia: Adicionar variação artificial às imagens para melhorar a generalização do modelo.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal_and_vertical"),  # Rotação aleatória em ambos os eixos
    layers.RandomRotation(0.05),  # Pequena rotação para evitar descaracterização
    layers.RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)),  # Zoom moderado
    layers.RandomCrop(img_height - 10, img_width - 10),  # Corte reduzido para preservar informações
    layers.Rescaling(1.0 / 255)  # Normalização dos valores de pixel para estabilidade numérica
  ]
)

### Definição do modelo sequencial
### Estratégia: Modelo simples com três camadas convolucionais intercaladas com pooling

In [ ]:
model = Sequential([
    input_layer,
    data_augmentation,
    layers.Conv2D(32, 3, padding='same', activation='relu'),  # Camada convolucional com 32 filtros
    layers.MaxPooling2D(),  # Redução de dimensionalidade
    layers.Conv2D(64, 3, padding='same', activation='relu'),  # Camada convolucional com 64 filtros
    layers.MaxPooling2D(),  # Redução de dimensionalidade
    layers.Conv2D(128, 3, padding='same', activation='relu'), # Camada convolucional com 128 filtros
    layers.MaxPooling2D(),  # Redução de dimensionalidade
    layers.Dropout(0.2),  # Dropout para evitar overfitting
    layers.Flatten(),  # Achatar a saída para entrada da camada densa
    layers.Dense(256, activation='relu'),  # Camada densa intermediária
    layers.Dropout(0.4),  # Dropout adicional para regularização
    layers.Dense(num_classes)  # Camada final para previsão das classes
])

### Compilação do modelo
### Estratégia: Adam otimiza com um learning rate inicial fixado, Crossentropy para problemas de classificação.

In [ ]:
optimizer = Adam(learning_rate=0.001)  # Learning rate ajustado para estabilidade
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Definição do número de épocas
### Estratégia: Número alto de épocas para permitir convergência gradual.
### Treinamento do modelo


In [ ]:
epochs = 150
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
)

Epoch 1/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 77s 239ms/step - accuracy: 0.1673 - loss: 2.5559 - val_accuracy: 0.3971 - val_loss: 1.7354
Epoch 2/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 23s 166ms/step - accuracy: 0.3953 - loss: 1.7385 - val_accuracy: 0.5117 - val_loss: 1.4324
Epoch 3/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 41s 166ms/step - accuracy: 0.4504 - loss: 1.5584 - val_accuracy: 0.5451 - val_loss: 1.2741
Epoch 4/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 23s 165ms/step - accuracy: 0.5030 - loss: 1.4002 - val_accuracy: 0.6011 - val_loss: 1.2122
Epoch 5/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 41s 165ms/step - accuracy: 0.5332 - loss: 1.3123 - val_accuracy: 0.6435 - val_loss: 1.0737
Epoch 6/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 41s 165ms/step - accuracy: 0.5927 - loss: 1.1748 - val_accuracy: 0.6543 - val_loss: 1.0234
Epoch 7/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 23s 163ms/step - accuracy: 0.6195 - loss: 1.0936 - val_accuracy: 0.6724 - val_loss: 0.9377
Epoch 8/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 23s 163ms/step - accuracy: 0.6307 -

## Arquitetura
As Redes Neurais Convolucionais (Convolutional Neural Networks - CNNs) são um tipo de arquitetura de rede neural projetada para processar dados que possuem uma estrutura de grade, como imagens.

### Camadas Convolucionais:
Extraem características das imagens, como bordas, texturas e padrões complexos.
Utilizam filtros (kernels) que percorrem a imagem aplicando convoluções.
### Camadas de Pooling:
Reduzem a dimensionalidade espacial (altura e largura) da imagem, preservando informações importantes.
A mais comum é o Max Pooling, que pega o valor máximo em uma região específica.
### Dropout:
Evita overfitting ao desligar aleatoriamente algumas conexões durante o treinamento.
### Camada Flatten:
Usada para achatar (converter) a saída das camadas anteriores, que geralmente possuem formato multidimensional (e.g., altura, largura e canais), em um vetor unidimensional.
### Camada Dense:
Conecta cada neurônio da entrada a cada neurônio da saída.

### Referências:
https://www.tensorflow.org/tutorials/images/cnn?hl=pt-br